In [7]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
import time
import pandas as pd
import os
import seaborn as sns
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.layers import MaxPool2D ,ReLU,Lambda,TimeDistributed,Dense, GlobalAveragePooling2D, Dropout,LSTM,Conv2D,MaxPooling2D,Flatten,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import CSVLogger

import os
import random

import numpy as np
import cv2

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

from PIL import Image, ImageOps
import matplotlib.image as mpimg

from tensorflow.keras.models import model_from_json

In [9]:
batch_size = 50
image_height=64
image_width=64
train_path = 'Dataset/Train/'     

test_path =  'Dataset/Test'  
train_datagenerator = ImageDataGenerator(
        shear_range=0.2,
#         zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255,
vertical_flip = True,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range=0.3,
    rotation_range=15,
)

test_datagenerator = ImageDataGenerator(rescale=1./255)
train_generator = train_datagenerator.flow_from_directory(train_path,
                                                          target_size = (64, 64),
                                                          color_mode = 'rgb',
                                                          batch_size = batch_size,
                                                          class_mode = 'categorical',
                                                          shuffle = True)


test_generator = test_datagenerator.flow_from_directory(test_path,
                                                        target_size = (64, 64),
                                                        color_mode = 'rgb',
                                                        class_mode = 'categorical',
                                                       shuffle=False
                                                       )

#Load model wothout classifier/fully connected layers
model = tf.keras.applications.VGG16(weights='imagenet',input_shape=(64,64,3), classes=10,include_top=False)

#for layer in model.layers:
#    layer.trainable = False
model.summary()

flat1 = Flatten()(model.layers[-1].output)

class1 = Dense(1024, activation='relu')(flat1)
# class2 = Dropout(0.2)(class1)
class3 = Dense(512, activation='relu')(class1)
# class4 = Dropout(0.2)(class3)
class5 = Dense(256, activation='relu')(class3)
# class6 = Dropout(0.2)(class5)
class7 = Dense(128, activation='relu')(class5)
# class8 = Dropout(0.2)(class7)
x = class7
output = Dense(10, activation='softmax')(x)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()
model.compile(optimizer = keras.optimizers.SGD(learning_rate=keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)),loss='categorical_crossentropy',metrics = ['accuracy'])

Found 3607 images belonging to 10 classes.
Found 400 images belonging to 10 classes.
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
__________________________________________

In [ ]:
csv_logger = CSVLogger('training_vgg16_rd_forest_character.log', separator=',', append=False)
history1=model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples//batch_size,
        epochs=20,
        validation_data=train_generator,
        validation_steps=train_generator.samples//batch_size,
        callbacks=[csv_logger]
)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

from matplotlib import pyplot as plt

plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model_pred = model.evaluate(test_generator)